# Cartpole DQN

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = ''

# Import dependencies

In [2]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import os

Using TensorFlow backend.


# Set parameters

In [3]:
env = gym.make('CartPole-v0') 

/home/sarbjeet/git/gym/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [4]:
state_size = env.observation_space.shape[0]
state_size

4

In [5]:
action_size = env.action_space.n
action_size

2

In [6]:
batch_size = 32

In [7]:
n_episodes = 1001

In [8]:
output_dir = 'model_Output/cartpole'

In [9]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Define Agent

In [10]:
class DQNAgent:
    
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        
        self.memory = deque(maxlen = 2000)
        
        self.gamma = 0.95
        
        self.epsilon = 1.0
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        
        self.learning_rate = 0.001
        
        self.model = self._build_model()
        
        
    def _build_model(self) :
        
        model = Sequential()
        
        model.add(Dense(24, input_dim = state_size, activation = 'relu'))
        model.add(Dense(24,activation='relu'))
        model.add(Dense(self.action_size, activation = 'linear'))
        
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate)) 
        
        return model
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        
    def act(self, state):
        
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])
    
    def replay(self, batch_size):
        
        minibatch = random.sample(self.memory,batch_size)
        
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma * np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            
            self.model.fit(state,target_f, epochs=1, verbose=0)
            
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
    
    def load (self,name):
        self.model.load_weights(name)
    
    def save (self,name):
        self.model.save_weights(name) 


In [11]:
agent = DQNAgent(state_size, action_size)

# Interact with Environment

In [12]:
done = False
for e in range(n_episodes):
    
    state = env.reset()
    state = np.reshape(state, [1,state_size])
    
    for time in range(5000):
        
        env.render() 
        action = agent.act(state) 
        
        next_state, reward, done, _ = env.step(action)
        
        reward = reward if not done else -10
        
        next_state = np.reshape(next_state, [1,state_size])
        
        agent.remember(state, action, reward, next_state, done) 
        
        state = next_state 
        
        if done:
            print("episode: {}/{}, score: {}, e: {:.2}".format(e, n_episodes, time, agent.epsilon))
            break
        
    if len(agent.memory) > batch_size:
        agent.replay(batch_size) 
    
    if e % 50 == 0:
        agent.save(output_dir + "weights_"+'{:04d}'.format(e)+'.hsf5')

episode: 0/1001, score: 23, e: 1.0
episode: 1/1001, score: 15, e: 1.0

episode: 2/1001, score: 12, e: 0.99
episode: 3/1001, score: 13, e: 0.99
episode: 4/1001, score: 20, e: 0.99
episode: 5/1001, score: 36, e: 0.98
episode: 6/1001, score: 63, e: 0.98
episode: 7/1001, score: 20, e: 0.97
episode: 8/1001, score: 27, e: 0.97
episode: 9/1001, score: 15, e: 0.96
episode: 10/1001, score: 26, e: 0.96
episode: 11/1001, score: 46, e: 0.95
episode: 12/1001, score: 32, e: 0.95
episode: 13/1001, score: 27, e: 0.94
episode: 14/1001, score: 14, e: 0.94
episode: 15/1001, score: 14, e: 0.93
episode: 16/1001, score: 9, e: 0.93


KeyboardInterrupt: 